<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>
SwissStore - TCO & EAC Calculation </p><br>

This notebook was created as a part of the SwissStore project and calculates the Total Cost of Ownership (TCO) and Equivalent Annual Cost (EAC) for a specific set of optimized scenario results.


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Importing Python libraries </p><br>

In [1]:
import pandas as pd
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import itertools
import gdxtools as gt


#Module that provides a portable way of using operating system dependent functionality
import os

from itertools import permutations, repeat

#Module to interact with the Notebook environment 
#https://ipywidgets.readthedocs.io/en/stable/
import ipywidgets as widgets

#Module for character encoding auto-detection 
#https://chardet.readthedocs.io/en/latest/index.html
import chardet 

#Plotting modules
import matplotlib
from matplotlib import pyplot as plt
#Allows plots to be shown on the Notebook
%matplotlib inline 
#%matplotlib notebook


#Data visualization module based on matplotlib
#https://seaborn.pydata.org/
import seaborn as sns

#Cross-platform colored terminal text 
#https://pypi.org/project/colorama/
from colorama import Fore
from colorama import Style

from pandas.io.parsers import ParserError

from itertools import product

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Importing data and creating dataframes
</p><br>

In [2]:
#dir_source = './Data/Dummy data_dummy_v8'
#dir_source = './Data/Dummy data_dummy_tco_test'

dir_source = './Data/Dummy data_dummy_100_test'
dir_parsed = './Results'

In [3]:
#Creating a list with the names of the data tables stored in the source directory
data_tables = os.listdir(dir_source)
# print(data_tables)

del_list = ['01.swisstore_table1_unige.csv',\
            '06.swisstore_table6_hslu.csv',\
            '07.swisstore_table7_hslu.csv','08.swisstore_table8_hslu.csv','09.swisstore_table9_hslu.csv',\
            '10.swisstore_table10_unige.xlsx','11.swisstore_table11_unige.csv',\
            '15.swisstore_table15_hslu.xlsx',\
            '16.swisstore_table16_hslu.xlsx','18.swissstore_table18_avg_spot_prices.xlsx',\
            '19.swissstore_table_19_etariff.xlsx']
print('del_list',del_list)

for i in del_list:
    print(i)
    data_tables.remove(i)

#Creating an empty dictionary to store the data from the csv files
tables = {}

key_names = ['2_master_table', '3_retrofits', '4_new_technologies', \
             '5_energy_costs','12_elec_dem_ind','13_base_energy_costs_scen',\
             '14_tax_scen','17_rcp_scen']

count = 0

#Looping through the csv files and uploading the data to the `Tables` dictionary
for fn in data_tables:
    try:
        name = key_names[count]
        path = dir_source + "/" + fn

        try:
            table_df = pd.read_csv(path, sep = "\,", engine='python')
            
            table_shape = table_df.shape
          
            if table_shape[1] == 1:
                table_df = pd.read_csv(path, sep = "\;", engine='python')
            else:
                pass
            
            tables[name] = table_df

        except ParserError:
            tables[name] = pd.read_excel(path,engine = 'xlrd')
        except IndexError:
            pass
    except IndexError:
        pass
        
        
    count += 1
    
print(data_tables)    
print(tables)

del_list ['01.swisstore_table1_unige.csv', '06.swisstore_table6_hslu.csv', '07.swisstore_table7_hslu.csv', '08.swisstore_table8_hslu.csv', '09.swisstore_table9_hslu.csv', '10.swisstore_table10_unige.xlsx', '11.swisstore_table11_unige.csv', '15.swisstore_table15_hslu.xlsx', '16.swisstore_table16_hslu.xlsx', '18.swissstore_table18_avg_spot_prices.xlsx', '19.swissstore_table_19_etariff.xlsx']
01.swisstore_table1_unige.csv
06.swisstore_table6_hslu.csv
07.swisstore_table7_hslu.csv
08.swisstore_table8_hslu.csv
09.swisstore_table9_hslu.csv
10.swisstore_table10_unige.xlsx
11.swisstore_table11_unige.csv
15.swisstore_table15_hslu.xlsx
16.swisstore_table16_hslu.xlsx
18.swissstore_table18_avg_spot_prices.xlsx
19.swissstore_table_19_etariff.xlsx
['02.swisstore_table2_unige.csv', '03.swisstore_table3_unige.csv', '04.swisstore_table4_hslu.xlsx', '05.swisstore_table5_hslu.xlsx', '12.swisstore_table12_unige.xlsx', '13.swisstore_table13_hslu.xlsx', '14.swisstore_table14_unibas.xlsx', '17.swisstore_table

In [4]:
#The function building_code is created to include the retrofit data from table_3 in table_2 
def building_code(df):
    """Returns a DataFrame with a column named 'build_code', aggregating the data from the 
    'constr_period', 'build_type' and 'area_typology' columns. The format is "year of construction-building type-typology"
    
    :param: df: <DataFrame> unedited table 2 or 3
    return: df: <DataFrame> Edited tables with the column 'build_code'"""
    
    df['build_code'] = df['constr_period'].map(str) + '-' + df['build_type'].map(str) + '-' \
                    + df['area_typology'].map(str)
    
    df['arch_typ'] = df['build_type'].map(str) + '-' + df['area_typology'].map(str)
                    
    
    #df = df.drop(columns = ['constr_period','build_type','area_typology'])
    
    #Moving the column 'build_code' to the first position
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    return df

In [5]:
def rename_cols(tables_key):#Creating table_n as a previous version of table_3 

    table_new = tables.get(tables_key)

    #Renaming table_n's columns to be consistent with table_2's column titles

    table_new.rename(columns={'age':'constr_period','buildingtype':'build_type',\
                        'typology':'area_typology'}, inplace=True)

    return(table_new)

In [6]:

#Creating a data frame for each table. For tables 2 and 3 the above function `archetype` 
#is applied
table_2 = building_code(tables.get('2_master_table'))
table_3 = building_code(rename_cols(tables_key = '3_retrofits'))
table_4 = tables.get('4_new_technologies')
table_5 = tables.get('5_energy_costs')
table_12 = tables.get('12_elec_dem_ind')
table_13 = tables.get('13_base_energy_costs_scen')
table_14 = tables.get('14_tax_scen')
# table_17 = tables.get('17_rcp_scen')
table_17 = building_code(rename_cols(tables_key ='17_rcp_scen'))

In [129]:
print(table_14)

   tax_type    scenario  2020      2035      2050
0   co2_tax         bau     1  1.357143  1.928571
1   co2_tax  optimistic     1  1.612500  2.187500
2   co2_tax   ambitious     1  3.071429  4.166667
3   mos_tax         bau     1  1.000000  1.000000
4   mos_tax  optimistic     1  1.000000  1.000000
5   mos_tax   ambitious     1  1.000000  1.000000
6  mwst_tax         bau     1  1.000000  1.000000
7  mwst_tax  optimistic     1  1.000000  1.000000
8  mwst_tax   ambitious     1  1.000000  1.000000


In [7]:
#print(table_1)
display(table_2)
# print(table_3)
# print(table_4)
# print(table_5)
# print(table_6)
# print(table_7)
# print(table_8)
# print(table_9)
# print(table_13)
# print(table_14)
display(table_17)

,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,constr_period,build_type,area_typology,canton,hvac_system,arch_entry_org_id,hvac_system_org,dhw_decentral,dhw_daily_energy,dhw_storage_decentral,dhw_power_decentral,p_therm_heating,enable_borehole,ht_temp_level,enable_gas,enable_district_heat,p_dhw_cent,p_therm_tot,sys_enable_dh,sys_enable_hpaw,sys_enable_hpbw,sys_enable_wood,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code
0,SFH-Rural,407,15023,110.0,110.000000,1,191,1920,SFH,Rural,NE,Oil,1030392,Oil,0,4.219178,0.000000,0.000000,14.139469,0,0,0,0,0.220000,14.359469,0,1,0,1,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural
1,MFH-Rural,3962,920,433.0,517.250000,4,78,1990,MFH,Rural,LU,Oil,1226166,Oil,0,29.759589,0.000000,0.000000,21.221423,1,1,0,1,1.551750,22.773173,1,1,1,1,0,0,1,3.694643,258.625000,107.333333,1226166,1990-MFH-Rural
2,SFH-Suburban,2511,6884,184.0,191.760000,16,115,1960,SFH,Suburban,TI,Oil,1148983,Oil,0,7.355178,0.000000,0.000000,13.045963,1,1,1,1,0.383520,13.429483,1,1,1,1,0,1,1,1.369714,95.880000,86.984211,1310946,1960-SFH-Suburban
3,SFH-Suburban,5663,5776,224.5,224.500000,2,57,2010,SFH,Suburban,FR,Gas,1300476,Gas,0,8.610959,0.000000,0.000000,6.965903,0,0,1,0,0.449000,7.414903,0,1,0,1,0,1,0,1.603571,62.860000,74.900000,1300476,2010-SFH-Suburban
4,SFH-Suburban,509,1665,138.0,154.909091,8,121,1920,SFH,Suburban,BL,Wood,1044460,Wood,0,5.941719,0.000000,0.000000,9.490558,0,1,1,1,0.309818,9.800377,1,1,0,1,0,1,0,1.106494,21.687273,92.506061,1320270,1920-SFH-Suburban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SFH-Rural,4281,18016,184.0,200.000000,3,106,1990,SFH,Rural,LU,Wood,1240710,Wood,1,7.671233,11.506849,1.917808,11.017110,1,1,0,1,0.000000,11.017110,1,1,1,1,0,0,0,1.428571,100.000000,87.500000,1240710,1990-SFH-Rural
96,MFH-Suburban,5389,5958,630.0,630.000000,1,42,2010,MFH,Suburban,SH,HPBW,1290392,HP,0,36.246575,0.000000,0.000000,14.958192,1,0,0,0,1.890000,16.848192,0,1,1,1,0,0,0,4.500000,176.400000,122.500000,1290392,2010-MFH-Suburban
97,SFH-Suburban,4461,6742,98.0,98.000000,1,68,1990,SFH,Suburban,TI,Electro,1252449,Electro,1,3.758904,5.638356,0.939726,4.025507,0,0,0,0,0.000000,4.025507,0,1,0,1,0,0,0,0.700000,49.000000,83.300000,1252449,1990-SFH-Suburban
98,SFH-Rural,1024,16200,114.0,114.000000,1,170,1945,SFH,Rural,BE,Wood,1069997,Wood,0,4.372603,0.000000,0.000000,11.722810,0,0,0,0,0.228000,11.950810,0,1,0,1,0,0,0,0.814286,57.000000,199.500000,1069997,1945-SFH-Rural


,arch_typ,rcp,constr_period,build_type,area_typology,canton,hvac_system_org,scenario_year,useful_energy__kwh_per_m2,era__m2,build_code
0,MFH-Rural,8.5,1920,MFH,Rural,SH,DH,2020,127.422429,4772,1920-MFH-Rural
1,MFH-Rural,2.6,1920,MFH,Rural,BE,DH,2020,142.044250,21700,1920-MFH-Rural
2,MFH-Rural,2.6,1920,MFH,Rural,BL,DH,2020,129.987921,15774,1920-MFH-Rural
3,MFH-Rural,2.6,1920,MFH,Rural,FR,DH,2020,121.300314,3532,1920-MFH-Rural
4,MFH-Rural,2.6,1920,MFH,Rural,GL,DH,2020,190.903501,757,1920-MFH-Rural
...,...,...,...,...,...,...,...,...,...,...,...
157387,SFH-Urban,8.5,2010,SFH,Urban,BE,Electro,2090,49.475081,801,2010-SFH-Urban
157388,SFH-Urban,8.5,2010,SFH,Urban,SZ,Electro,2090,55.048837,2209,2010-SFH-Urban
157389,SFH-Urban,8.5,2010,SFH,Urban,TI,Electro,2090,46.152105,6623,2010-SFH-Urban
157390,SFH-Urban,8.5,2010,SFH,Urban,VS,Electro,2090,50.413620,4443,2010-SFH-Urban


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Climate scenarios and reference year
</p><br>

In [8]:
rcp_scen_list = pd.unique(table_17["rcp"]).tolist()
rcp_year_list = pd.unique(table_17["scenario_year"]).tolist()
rcp_scenario_year_list = [2020,2025,2030,2035,2040,2045,2050]

In [9]:
dir_pv_scen = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/scen_results'
pv_scen_list = os.listdir(dir_pv_scen)
pv_scen_list

['pv_opt', 'pv_zro']

In [10]:
pv_scen_widget = widgets.Dropdown(
    options = pv_scen_list,
    description = 'pv_scen ',
    disabled = False)
display(pv_scen_widget)

Dropdown(description='pv_scen ', options=('pv_opt', 'pv_zro'), value='pv_opt')

In [11]:
pv_scen = pv_scen_widget.value
pv_scen

'pv_zro'

In [12]:
dir_results = dir_pv_scen + '/' + pv_scen
data_results_list = os.listdir(dir_results)

In [13]:
data_results_widget = widgets.Dropdown(
    options = data_results_list,
    description = 'Data set ',
    disabled = False)
display(data_results_widget)

Dropdown(description='Data set ', options=('2020_rcp26_Low_100_results_pv_zro.gdx', '2020_rcp85_Low_100_result…

In [14]:
data_results = data_results_widget.value
data_results

'2050_rcp85_Low_100_results_pv_zro.gdx'

In [15]:
rcp_scenario_year = int(data_results[:4])
rcp_file_name = data_results[5:10]

In [16]:
rcp_file_name

'rcp85'

In [17]:
if rcp_file_name == 'rcp85':
    rcp_scen = 8.5
if rcp_file_name == 'rcp45':
    rcp_scen = 4.5
if rcp_file_name == 'rcp26':
    rcp_scen = 2.6
    
rcp_scen

8.5

In [18]:
pd.options.mode.chained_assignment = None
table_2.loc[:,'rcp'] = rcp_scen
table_2.loc[:,'scenario_year'] = rcp_scenario_year
table_2.head(10)

,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,constr_period,build_type,area_typology,canton,hvac_system,arch_entry_org_id,hvac_system_org,dhw_decentral,dhw_daily_energy,dhw_storage_decentral,dhw_power_decentral,p_therm_heating,enable_borehole,ht_temp_level,enable_gas,enable_district_heat,p_dhw_cent,p_therm_tot,sys_enable_dh,sys_enable_hpaw,sys_enable_hpbw,sys_enable_wood,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code,rcp,scenario_year
0,SFH-Rural,407,15023,110.0,110.000000,1,191,1920,SFH,Rural,NE,Oil,1030392,Oil,0,4.219178,0.000000,0.000000,14.139469,0,0,0,0,0.220000,14.359469,0,1,0,1,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural,8.5,2050
1,MFH-Rural,3962,920,433.0,517.250000,4,78,1990,MFH,Rural,LU,Oil,1226166,Oil,0,29.759589,0.000000,0.000000,21.221423,1,1,0,1,1.551750,22.773173,1,1,1,1,0,0,1,3.694643,258.625000,107.333333,1226166,1990-MFH-Rural,8.5,2050
2,SFH-Suburban,2511,6884,184.0,191.760000,16,115,1960,SFH,Suburban,TI,Oil,1148983,Oil,0,7.355178,0.000000,0.000000,13.045963,1,1,1,1,0.383520,13.429483,1,1,1,1,0,1,1,1.369714,95.880000,86.984211,1310946,1960-SFH-Suburban,8.5,2050
3,SFH-Suburban,5663,5776,224.5,224.500000,2,57,2010,SFH,Suburban,FR,Gas,1300476,Gas,0,8.610959,0.000000,0.000000,6.965903,0,0,1,0,0.449000,7.414903,0,1,0,1,0,1,0,1.603571,62.860000,74.900000,1300476,2010-SFH-Suburban,8.5,2050
4,SFH-Suburban,509,1665,138.0,154.909091,8,121,1920,SFH,Suburban,BL,Wood,1044460,Wood,0,5.941719,0.000000,0.000000,9.490558,0,1,1,1,0.309818,9.800377,1,1,0,1,0,1,0,1.106494,21.687273,92.506061,1320270,1920-SFH-Suburban,8.5,2050
5,SFH-Rural,4945,9704,183.0,183.000000,2,77,2000,SFH,Rural,SG,Wood,1272573,Wood,0,7.019178,0.000000,0.000000,7.552604,0,0,0,0,0.366000,7.918604,0,1,0,1,0,0,0,1.307143,51.240000,75.250000,1272573,2000-SFH-Rural,8.5,2050
6,SFH-Suburban,4468,10500,146.0,146.000000,1,77,1990,SFH,Suburban,VD,Gas,1253269,Gas,0,5.600000,0.000000,0.000000,7.361722,0,0,1,0,0.292000,7.653722,0,1,0,1,0,1,0,1.042857,73.000000,67.200000,1253269,1990-SFH-Suburban,8.5,2050
7,SFH-Rural,3588,7370,126.0,170.666667,3,125,1980,SFH,Rural,BE,Electro,1198268,Electro,1,6.546119,9.819178,1.636530,10.920391,1,0,0,1,0.000000,10.920391,1,1,1,1,0,0,0,1.219048,42.666667,44.333333,1198268,1980-SFH-Rural,8.5,2050
8,SFH-Rural,4346,10523,195.0,195.000000,1,105,1990,SFH,Rural,VS,Electro,1245415,Electro,1,7.479452,11.219178,1.869863,12.711258,0,0,0,0,0.000000,12.711258,0,1,0,1,0,0,0,1.392857,97.500000,39.666667,1245415,1990-SFH-Rural,8.5,2050
9,SFH-Urban,1952,4026,130.0,130.000000,1,133,1950,SFH,Urban,SG,Wood,1122003,Wood,0,4.986301,0.000000,0.000000,10.713542,0,0,0,0,0.260000,10.973542,0,1,0,1,0,0,0,0.928571,26.000000,49.000000,1122003,1950-SFH-Urban,8.5,2050


In [19]:
# table_2[table_2.arch_entry_id == 1017477]

In [20]:
#The function building_code is created to include the retrofit data from table_3 in table_2 
def rcp_code(df):
    """Returns a DataFrame with a column named 'build_code', aggregating the data from the 
    'constr_period', 'build_type' and 'area_typology' columns. The format is "year of construction-building type-typology"
    
    :param: df: <DataFrame> unedited table 2 or 3
    return: df: <DataFrame> Edited tables with the column 'build_code'"""
    
    df['rcp_code'] = df['build_code'].map(str) + '-' + df['canton'].map(str) + '-' + \
    df['hvac_system_org'].map(str) + '-' + df['rcp'].map(str)
                    
    
    #df = df.drop(columns = ['constr_period','build_type','area_typology'])
    
    #Moving the column 'build_code' to the first position
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    return df

In [21]:
table_17 = rcp_code(table_17)
table_2 = rcp_code(table_2)

In [22]:
"Calculating the new heating demand based on the chosen climate scenario"

rcp_code_list = pd.unique(table_2['rcp_code']).tolist()

table_2_rcp = table_2.copy()
master_rcp = pd.DataFrame()


for rcp_code in rcp_code_list:

    table_17_rcp = table_17.loc[table_17['rcp_code']== rcp_code]

    for rcp_year in rcp_scenario_year_list:
        if rcp_year not in rcp_year_list:
            table_17_rcp.loc[-1,'scenario_year'] = rcp_year
        else:
            pass
        table_17_rcp = table_17_rcp.sort_values('scenario_year').reset_index(drop=True)

        table_17_rcp = table_17_rcp.interpolate(method ='linear', limit_direction ='forward')
    
#     display('table_17_rcp',table_17_rcp)
#     display('rcp_scenario_year',rcp_scenario_year)
    
    
    energy_specific_base = table_17_rcp.loc[table_17_rcp['scenario_year'] == 2020, 'useful_energy__kwh_per_m2'].item()
    energy_specific_scen = table_17_rcp.loc[table_17_rcp['scenario_year'] == rcp_scenario_year, 'useful_energy__kwh_per_m2'].item()

#     display('base',energy_specific_base)
#     display('scen',energy_specific_scen)
    rcp_factor = energy_specific_scen / energy_specific_base
    
#     display('rcp_factor',rcp_factor)
    
    table_2_rcp_filt = table_2_rcp[table_2_rcp['rcp_code']== rcp_code]
#     display('table_2_rcp_filt_orig',table_2_rcp_filt)
    table_2_rcp_filt['energy_specific'] = table_2_rcp_filt['energy_specific'] * rcp_factor
#     display('table_2_rcp_filt_new',table_2_rcp_filt)
    
    master_rcp = master_rcp.append(table_2_rcp_filt)
    
table_2 = master_rcp

display(table_2)

,rcp_code,arch_typ,arch_id,clim_id,era_median,era_avg,tot_buildings,energy_specific,constr_period,build_type,area_typology,canton,hvac_system,arch_entry_org_id,hvac_system_org,dhw_decentral,dhw_daily_energy,dhw_storage_decentral,dhw_power_decentral,p_therm_heating,enable_borehole,ht_temp_level,enable_gas,enable_district_heat,p_dhw_cent,p_therm_tot,sys_enable_dh,sys_enable_hpaw,sys_enable_hpbw,sys_enable_wood,sys_enable_electro,sys_enable_gas,sys_enable_oil,n_dhw_cent,roof_era,era_roof_new,arch_entry_id,build_code,rcp,scenario_year
0,1920-SFH-Rural-NE-Oil-8.5,SFH-Rural,407,15023,110.0,110.000000,1,169.932684,1920,SFH,Rural,NE,Oil,1030392,Oil,0,4.219178,0.000000,0.000000,14.139469,0,0,0,0,0.220000,14.359469,0,1,0,1,0,0,1,0.785714,15.400000,47.600000,1030392,1920-SFH-Rural,8.5,2050
1,1990-MFH-Rural-LU-Oil-8.5,MFH-Rural,3962,920,433.0,517.250000,4,70.674780,1990,MFH,Rural,LU,Oil,1226166,Oil,0,29.759589,0.000000,0.000000,21.221423,1,1,0,1,1.551750,22.773173,1,1,1,1,0,0,1,3.694643,258.625000,107.333333,1226166,1990-MFH-Rural,8.5,2050
2,1960-SFH-Suburban-TI-Oil-8.5,SFH-Suburban,2511,6884,184.0,191.760000,16,100.262946,1960,SFH,Suburban,TI,Oil,1148983,Oil,0,7.355178,0.000000,0.000000,13.045963,1,1,1,1,0.383520,13.429483,1,1,1,1,0,1,1,1.369714,95.880000,86.984211,1310946,1960-SFH-Suburban,8.5,2050
3,2010-SFH-Suburban-FR-Gas-8.5,SFH-Suburban,5663,5776,224.5,224.500000,2,50.844244,2010,SFH,Suburban,FR,Gas,1300476,Gas,0,8.610959,0.000000,0.000000,6.965903,0,0,1,0,0.449000,7.414903,0,1,0,1,0,1,0,1.603571,62.860000,74.900000,1300476,2010-SFH-Suburban,8.5,2050
4,1920-SFH-Suburban-BL-Wood-8.5,SFH-Suburban,509,1665,138.0,154.909091,8,105.863005,1920,SFH,Suburban,BL,Wood,1044460,Wood,0,5.941719,0.000000,0.000000,9.490558,0,1,1,1,0.309818,9.800377,1,1,0,1,0,1,0,1.106494,21.687273,92.506061,1320270,1920-SFH-Suburban,8.5,2050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1990-SFH-Rural-LU-Wood-8.5,SFH-Rural,4281,18016,184.0,200.000000,3,95.695702,1990,SFH,Rural,LU,Wood,1240710,Wood,1,7.671233,11.506849,1.917808,11.017110,1,1,0,1,0.000000,11.017110,1,1,1,1,0,0,0,1.428571,100.000000,87.500000,1240710,1990-SFH-Rural,8.5,2050
96,2010-MFH-Suburban-SH-HP-8.5,MFH-Suburban,5389,5958,630.0,630.000000,1,38.619929,2010,MFH,Suburban,SH,HPBW,1290392,HP,0,36.246575,0.000000,0.000000,14.958192,1,0,0,0,1.890000,16.848192,0,1,1,1,0,0,0,4.500000,176.400000,122.500000,1290392,2010-MFH-Suburban,8.5,2050
97,1990-SFH-Suburban-TI-Electro-8.5,SFH-Suburban,4461,6742,98.0,98.000000,1,59.915467,1990,SFH,Suburban,TI,Electro,1252449,Electro,1,3.758904,5.638356,0.939726,4.025507,0,0,0,0,0.000000,4.025507,0,1,0,1,0,0,0,0.700000,49.000000,83.300000,1252449,1990-SFH-Suburban,8.5,2050
98,1945-SFH-Rural-BE-Wood-8.5,SFH-Rural,1024,16200,114.0,114.000000,1,151.614588,1945,SFH,Rural,BE,Wood,1069997,Wood,0,4.372603,0.000000,0.000000,11.722810,0,0,0,0,0.228000,11.950810,0,1,0,1,0,0,0,0.814286,57.000000,199.500000,1069997,1945-SFH-Rural,8.5,2050


In [23]:
# table_2[table_2.arch_entry_id == 1017477]

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Technology lists
</p><br>

In [24]:
#Creating a dataframe with system solution's id and technology types 
syst_sol_matrix = table_4

#Removing unnecesary rows and columns
#syst_sol_matrix = syst_sol_matrix.drop(columns = ['data_type','unit','parameter'])
syst_sol_tech = syst_sol_matrix.iloc[5:13]
syst_sol_decentral = syst_sol_matrix.iloc[13:14]
# print('syst_sol_tech',syst_sol_tech)
# print(syst_sol_decentral)

#Creating a list of system solution technologies
syst_sol_list = list(syst_sol_matrix.columns)

syst_sol_list.remove('th_syssolution_id')
# print(syst_sol_list)

In [25]:
def th_tech_list(syst_sol_df, syst_sol_list, tech_name):

    tech_list = []

    for syst in syst_sol_list:
        if tech_name != 'th_dhw_decentral':
            syst_sol_df = syst_sol_tech.copy()
        else:
            syst_sol_df = syst_sol_decentral.copy()
        
        
        syst_sol_df  = syst_sol_df[['th_syssolution_id', syst]]
            
        syst_sol_df.rename(columns={syst:'tech'}, inplace=True)
 
        syst_sol_df  = syst_sol_df[(syst_sol_df.th_syssolution_id == tech_name) & (syst_sol_df.tech == 1.0)]
            
        if syst_sol_df.empty:
            pass
            
        else:
            tech_list.append(syst)

    return(tech_list)

In [26]:
#Creating lists for each system solution technology using the 'th_tech_list' function
syst_sol_dict = {}
syst_sol_dict['th_dh_list']= th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_dh') #District heating

syst_sol_dict['th_hpaw_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpaw') #Air water heat pump
syst_sol_dict['th_hpbw_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpbw') #Brine water heat pump
syst_sol_dict['th_wood_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_wood') #Wood
syst_sol_dict['th_electro_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_electro') #Electric heaters
syst_sol_dict['th_gas_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_gas') #Gas
syst_sol_dict['th_oil_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_oil') #Oil
syst_sol_dict['th_stes_list'] = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_stes') #stes
syst_sol_dict['th_decentral_list'] = th_tech_list(syst_sol_decentral,syst_sol_list,'th_dhw_decentral') #Decentral

th_dh_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_dh') #District heating
th_hpaw_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpaw') #Air water heat pump
th_hpbw_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_hpbw') #Brine water heat pump
th_wood_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_wood') #Wood
th_electro_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_electro') #Electric heaters
th_gas_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_gas') #Gas
th_oil_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_oil') #Oil
th_stes_list = th_tech_list(syst_sol_tech,syst_sol_list,'th_systemtype_stes') #stes
th_decentral_list = th_tech_list(syst_sol_decentral,syst_sol_list,'th_dhw_decentral') #Decentral

# print(th_dh_list)
# print(th_hpaw_list)
# print(th_wood_list)
# print(th_electro_list)
# print(th_gas_list)
# print(th_oil_list)
# print(th_decentral_list)

print(syst_sol_dict)

{'th_dh_list': [], 'th_hpaw_list': [100066, 100067, 100068, 100069, 100070, 100071, 100072, 100073, 100074, 100075, 100076, 100077, 100078, 100079, 100080, 100081, 100082, 100083, 100084, 100085, 100123, 100124, 100125, 100126, 100127, 100128, 100129, 100130, 100131, 100132, 100133, 100134, 100135, 100136, 100137, 100138, 100139, 100140, 100141, 100142], 'th_hpbw_list': [100086, 100087, 100088, 100089, 100090, 100091, 100092, 100093, 100094, 100095, 100096, 100097, 100098, 100099, 100100, 100101, 100102, 100103, 100104, 100105, 100106, 100107, 100108, 100109, 100110, 100143, 100144, 100145, 100146, 100147, 100148, 100149, 100150, 100151, 100152, 100153, 100154, 100155, 100156, 100157, 100158, 100159, 100160, 100161, 100162, 100163, 100164, 100165, 100166, 100167], 'th_wood_list': [100041, 100042, 100043, 100044, 100045, 100046, 100047, 100048, 100049, 100050, 100051, 100052, 100053, 100054, 100055, 100056, 100057, 100058, 100059, 100060, 100061, 100062, 100063, 100064, 100065], 'th_ele

In [27]:
th_elec_list = th_hpaw_list + th_hpbw_list + th_electro_list
th_fuel_list = th_wood_list + th_gas_list + th_oil_list

In [28]:
"Creating the WACC dataframe to include the discount of the system's costs"

#Weighted average cost of capital
wacc = float(0.04)
print(wacc)
#Lifetime in years
lifetime = int(30)
start_year = int(2020)
end_year = start_year + lifetime + 1

discount_df = pd.DataFrame()

for year in range(start_year, end_year):
    
    discount_df[year] = [1 / ((1 + wacc))**(year - start_year)]
                             
print(discount_df)

0.04
   2020      2021      2022      2023      2024      2025      2026      2027     2028      2029      2030      2031      2032      2033      2034      2035      2036      2037      2038      2039      2040      2041      2042      2043      2044      2045      2046      2047      2048      2049      2050
0   1.0  0.961538  0.924556  0.888996  0.854804  0.821927  0.790315  0.759918  0.73069  0.702587  0.675564  0.649581  0.624597  0.600574  0.577475  0.555265  0.533908  0.513373  0.493628  0.474642  0.456387  0.438834  0.421955  0.405726  0.390121  0.375117  0.360689  0.346817  0.333477  0.320651  0.308319


In [29]:
# pv_scen = 'pvopt'
# pv_scen

In [30]:
data_results_path = dir_results + '/' + data_results
print(data_results_path)

C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/scen_results/pv_zro/2050_rcp85_Low_100_results_pv_zro.gdx


In [31]:
gdxin = gt.gdxrw.gdxReader(data_results_path)

In [32]:
gdx_stes_map_inst = gdxin.rgdx(name='map_scenarios')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
gdx_stes_map_inst

{'type': 'GamsSet',
 'dimension': 6,
 'domain': ['full_scen', 'a', 'h', 'pv', 'b', 's'],
 'number_records': 801,
 'text': 'all possible scenarios',
 'elements': [('1030392_100056_PV_LiB_100000',
   '1030392',
   '100056',
   'PV',
   'LiB',
   '100000'),
  ('1030392_100032_PV_LiB_100000', '1030392', '100032', 'PV', 'LiB', '100000'),
  ('1030392_100128_PV_LiB_100000', '1030392', '100128', 'PV', 'LiB', '100000'),
  ('1030392_100147_PV_LiB_100000', '1030392', '100147', 'PV', 'LiB', '100000'),
  ('1133034_100056_PV_LiB_100000', '1133034', '100056', 'PV', 'LiB', '100000'),
  ('1133034_100032_PV_LiB_100000', '1133034', '100032', 'PV', 'LiB', '100000'),
  ('1133034_100125_PV_LiB_100000', '1133034', '100125', 'PV', 'LiB', '100000'),
  ('1133034_100145_PV_LiB_100000', '1133034', '100145', 'PV', 'LiB', '100000'),
  ('1107559_100056_PV_LiB_100000', '1107559', '100056', 'PV', 'LiB', '100000'),
  ('1107559_100032_PV_LiB_100000', '1107559', '100032', 'PV', 'LiB', '100000'),
  ('1107559_100130_PV_LiB

In [33]:
gdx_stes_map = pd.DataFrame(data=gdx_stes_map_inst['elements'], 
                             columns=gdx_stes_map_inst['domain']) #Getting a specific DataFrame from the dictionary

gdx_stes_map['scen'] = gdx_stes_map['a'] + '_' + gdx_stes_map['h']

gdx_stes_map[['h','s']] = gdx_stes_map[['h','s']].apply(pd.to_numeric)

gdx_stes_map

,full_scen,a,h,pv,b,s,scen
0,1030392_100056_PV_LiB_100000,1030392,100056,PV,LiB,100000,1030392_100056
1,1030392_100032_PV_LiB_100000,1030392,100032,PV,LiB,100000,1030392_100032
2,1030392_100128_PV_LiB_100000,1030392,100128,PV,LiB,100000,1030392_100128
3,1030392_100147_PV_LiB_100000,1030392,100147,PV,LiB,100000,1030392_100147
4,1133034_100056_PV_LiB_100000,1133034,100056,PV,LiB,100000,1133034_100056
...,...,...,...,...,...,...,...
796,1316897_100144_PV_LiB_807595,1316897,100144,PV,LiB,807595,1316897_100144
797,1290392_100128_PV_LiB_819464,1290392,100128,PV,LiB,819464,1290392_100128
798,1290392_100148_PV_LiB_819464,1290392,100148,PV,LiB,819464,1290392_100148
799,1292871_100135_PV_LiB_863479,1292871,100135,PV,LiB,863479,1292871_100135


In [34]:
gdx_arch_list = (pd.unique(gdx_stes_map['a'])).tolist()
gdx_scen_list = (pd.unique(gdx_stes_map['scen'])).tolist()
gdx_stes_list = (pd.unique(gdx_stes_map['s'])).tolist()

In [35]:
def gdx_dict(gdx_key_list,gdx_map,dict_key,dict_value,tech_list):
    '''dict_key: a or scen
       dict_value: h or s'''
    
    gdx_dict = {}
    
    for x in gdx_key_list:

        gdx_map_df = gdx_map.loc[gdx_map[dict_key] == x]
        gdx_value_list = (pd.unique(gdx_map_df[dict_value])).tolist()
        gdx_value_list = list(set(tech_list)&set(gdx_value_list))
            
        gdx_dict[x] = gdx_value_list
        
    return(gdx_dict)


In [36]:
syst_elec_dict =\
      gdx_dict(gdx_key_list = gdx_arch_list,\
               gdx_map = gdx_stes_map,\
               dict_key = 'a',\
               dict_value = 'h',\
               tech_list = th_elec_list)
display(syst_elec_dict)

{'1030392': [100128, 100147],
 '1133034': [100145, 100125],
 '1107559': [100130, 100150],
 '1132705': [100147, 100127],
 '1069997': [100146, 100126],
 '1165553': [100146, 100126],
 '1021432': [100129, 100148],
 '1136584': [100147, 100127],
 '1206896': [100089, 100069],
 '1023478': [100145, 100125],
 '1107180': [100091, 100071],
 '1110917': [100089, 100069],
 '1118480': [100146, 100126],
 '1059152': [100132, 100151],
 '1101288': [100073, 100092],
 '1025186': [100130, 100150],
 '1035864': [100146, 100126],
 '1084861': [100128, 100148],
 '1116978': [100129, 100148],
 '1026107': [100072, 100091],
 '1110364': [100128, 100147],
 '1031611': [100145, 100125],
 '1192251': [100096, 100078],
 '1173736': [100154, 100135],
 '1065871': [100090, 100070],
 '1031558': [100129, 100148],
 '1116960': [100130, 100149],
 '1105291': [100128, 100148],
 '1003479': [100075, 100093],
 '1033028': [100129, 100148],
 '1017477': [100088, 100068],
 '1122003': [100146, 100126],
 '1053713': [100147, 100127],
 '1119862'

In [37]:
syst_fuel_dict =\
      gdx_dict(gdx_key_list = gdx_arch_list,\
               gdx_map = gdx_stes_map,\
               dict_key = 'a',\
               dict_value = 'h',\
               tech_list = th_fuel_list)
display(syst_fuel_dict)

{'1030392': [100056, 100032],
 '1133034': [100056, 100032],
 '1107559': [100056, 100032],
 '1132705': [100056, 100032],
 '1069997': [100056, 100032],
 '1165553': [100056, 100032],
 '1021432': [100056, 100032],
 '1136584': [100056, 100032],
 '1206896': [100001, 100042, 100017],
 '1023478': [100056, 100032],
 '1107180': [100001, 100018, 100044],
 '1110917': [100001, 100042, 100017],
 '1118480': [100056, 100032],
 '1059152': [100056, 100032],
 '1101288': [100001, 100018, 100043],
 '1025186': [100056, 100032],
 '1035864': [100056, 100032],
 '1084861': [100056, 100032],
 '1116978': [100056, 100032],
 '1026107': [100001, 100043, 100017],
 '1110364': [100056, 100032],
 '1031611': [100056, 100032],
 '1192251': [100003, 100019, 100047],
 '1173736': [100056, 100032],
 '1065871': [100001, 100043, 100017],
 '1031558': [100056, 100032],
 '1116960': [100056, 100032],
 '1105291': [100056, 100032],
 '1003479': [100018, 100002, 100044],
 '1033028': [100056, 100032],
 '1017477': [100001, 100042, 100017]

In [38]:
th_all_syst_list = th_elec_list + th_fuel_list
syst_all_dict =\
      gdx_dict(gdx_key_list = gdx_arch_list,\
               gdx_map = gdx_stes_map,\
               dict_key = 'a',\
               dict_value = 'h',\
               tech_list = th_all_syst_list)
display(syst_all_dict)

{'1030392': [100056, 100128, 100147, 100032],
 '1133034': [100056, 100145, 100125, 100032],
 '1107559': [100056, 100130, 100150, 100032],
 '1132705': [100056, 100147, 100032, 100127],
 '1069997': [100056, 100146, 100126, 100032],
 '1165553': [100056, 100146, 100126, 100032],
 '1021432': [100056, 100129, 100148, 100032],
 '1136584': [100056, 100147, 100032, 100127],
 '1206896': [100001, 100069, 100042, 100017, 100089],
 '1023478': [100056, 100145, 100125, 100032],
 '1107180': [100001, 100071, 100044, 100018, 100091],
 '1110917': [100001, 100069, 100042, 100017, 100089],
 '1118480': [100056, 100146, 100126, 100032],
 '1059152': [100056, 100132, 100032, 100151],
 '1101288': [100001, 100073, 100043, 100018, 100092],
 '1025186': [100056, 100130, 100150, 100032],
 '1035864': [100056, 100146, 100126, 100032],
 '1084861': [100056, 100128, 100148, 100032],
 '1116978': [100056, 100129, 100148, 100032],
 '1026107': [100001, 100072, 100043, 100017, 100091],
 '1110364': [100056, 100128, 100147, 100

In [39]:
syst_stes_dict =\
      gdx_dict(gdx_key_list = gdx_scen_list,\
               gdx_map = gdx_stes_map,\
               dict_key = 'scen',\
               dict_value = 's',\
               tech_list = gdx_stes_list)
display(syst_stes_dict)

{'1030392_100056': [100000],
 '1030392_100032': [100000],
 '1030392_100128': [100000, 203738, 814954],
 '1030392_100147': [100000, 203738, 814954],
 '1133034_100056': [100000],
 '1133034_100032': [100000],
 '1133034_100125': [100000, 202245, 808983],
 '1133034_100145': [100000, 202245, 808983],
 '1107559_100056': [100000],
 '1107559_100032': [100000],
 '1107559_100130': [100000, 206745, 826980],
 '1107559_100150': [100000, 206745, 826980],
 '1132705_100056': [100000],
 '1132705_100032': [100000],
 '1132705_100127': [100000, 203743, 814975],
 '1132705_100147': [100000, 203743, 814975],
 '1069997_100056': [100000],
 '1069997_100032': [100000],
 '1069997_100126': [100000, 813827, 203456],
 '1069997_100146': [100000, 813827, 203456],
 '1165553_100056': [100000],
 '1165553_100032': [100000],
 '1165553_100126': [100000, 812354, 203088],
 '1165553_100146': [100000, 812354, 203088],
 '1021432_100056': [100000],
 '1021432_100032': [100000],
 '1021432_100129': [100000, 823017, 205754],
 '1021432

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Wood and fossil fuel demand and cost calculation 
</p><br>

In [40]:
def discount_func(waac,start_yr,lifetime):
    
    end_yr = start_yr + lifetime + 1
    
    discount_df = pd.DataFrame()
    
    for yr in range(start_yr,end_yr):
        
        discount_df[yr] = [1 / ((1 + wacc))**(yr - start_yr)]
        
    return(discount_df)  

In [41]:
def alpha_func(wacc,lifetime):
    
    try:
    
        alpha = wacc/(1-((1+wacc)**(-lifetime))) 
    
        return(alpha)

    except ZeroDivisionError:
        return(0)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Electricity demand and cost calculation 
</p><br>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Importing GDX data
</p><br>

In [42]:
gdx_stes_par_inst = gdxin.rgdx(name='stes_par')
gdx_stes_par_inst
gdx_stes_par = pd.DataFrame(data=gdx_stes_par_inst['values'].keys(), 
                            columns= gdx_stes_par_inst['domain'])

gdx_stes_par['stes_par'] = gdx_stes_par_inst['values'].values()
gdx_stes_par = gdx_stes_par.round(4) #rounding the values to 3 decimals
gdx_stes_par.columns = ['s','var','stes_par']
# gdx_stes_par.rename(columns = {gdx_stes_par.columns[1]:'s'},inplace = True)
# gdx_stes_par.rename(columns = {'*':'var'},inplace = True)


stes_id_list = list(pd.unique(gdx_stes_par['s']))

var_list = list(pd.unique(gdx_stes_par['var']))

gdx_var = pd.DataFrame()
        
for stes_id in stes_id_list:

    gdx_stes = gdx_stes_par.copy()
    gdx_stes = gdx_stes[gdx_stes.s == stes_id].drop(['s'], axis=1).set_index('var').T.reset_index(drop = True)
    gdx_stes['s'] = stes_id

    gdx_var = pd.concat([gdx_var, gdx_stes], ignore_index = True)

gdx_var = gdx_var.rename_axis(None, axis = 1)
gdx_var = gdx_var.round(3)


gdx_stes_par = gdx_var.copy()
gdx_stes_par

,th_stes_emax1,th_stes_emax2,th_stes_emax3,th_stes_chargeefficiency3,s,th_stes_volume,th_stes_relativedailyloss,th_stes_placedinsidehouse,th_stes_ambienttemperature,stes_loss_tot_th,stes_loss_tot_e1_th,stes_loss_tot_e2_th,stes_loss_tot_e3_th,th_stes_tmax,th_stes_dtmax,th_stes_emaxtot,th_stes_pmaxcharge3,th_stes_socmin,stes_eac_capex
0,0.010,0.010,0.010,1.0,100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93.275,155.458,186.550,1.0,100111,5.330,0.178,1.0,20.0,0.520,0.112,0.297,0.520,90.0,70.0,435.283,2.000,0.11,925.118
2,109.375,182.292,218.750,1.0,100112,6.250,0.193,1.0,20.0,0.564,0.121,0.322,0.564,90.0,70.0,510.417,2.000,0.11,983.495
3,127.750,212.917,255.500,1.0,100113,7.300,0.209,1.0,20.0,0.609,0.130,0.348,0.609,90.0,70.0,596.167,3.000,0.11,1018.034
4,146.475,244.125,292.950,1.0,100114,8.370,0.224,1.0,20.0,0.652,0.140,0.373,0.652,90.0,70.0,683.550,4.000,0.11,1052.584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,1956.816,3261.269,3913.541,1.0,809131,111.815,0.817,1.0,20.0,2.384,0.511,1.362,2.384,90.0,70.0,9131.626,8.338,0.11,5592.516
202,2168.062,3613.335,4336.023,1.0,810117,123.886,0.860,1.0,20.0,2.510,0.538,1.434,2.510,90.0,70.0,10117.420,8.515,0.11,5972.232
203,1627.706,2712.767,3255.335,1.0,807595,93.010,0.746,1.0,20.0,2.174,0.466,1.242,2.174,90.0,70.0,7595.807,8.019,0.11,4970.038
204,4171.036,6951.532,8341.877,1.0,819464,238.339,1.193,1.0,20.0,3.481,0.746,1.989,3.481,90.0,70.0,19464.444,9.648,0.11,9083.103


In [43]:
def gdx_to_df(report_name,par_name):
    gdx_inst = gdxin.rgdx(name= report_name) #creates a data instance
    gdx_df = pd.DataFrame(data=gdx_inst['values'].keys(), 
                             columns=gdx_inst['domain']) #Getting a specific DataFrame from the dictionary
    
    gdx_df[par_name] = gdx_inst['values'].values() #Renaming the parameter column
    
    gdx_df.rename(columns = {'*':'var'},inplace = True) 
    
    full_scen_list = list(pd.unique(gdx_df['full_scen']))
    
    var_list = list(pd.unique(gdx_df['var']))
    
    if len(var_list) == 1:
    
        del gdx_df['var']
        
        gdx_df = gdx_df.round(3) #rounding the values to 3 decimals
        
        return(gdx_df)
    
    else:
        gdx_var = pd.DataFrame()
        
        for scen in full_scen_list:

            gdx_scen = gdx_df.copy()
            gdx_scen = gdx_scen[gdx_scen.full_scen == scen].drop(['full_scen'], axis=1).set_index('var').T.reset_index(drop = True)
            gdx_scen['full_scen'] = scen
            
            gdx_var = pd.concat([gdx_var, gdx_scen], ignore_index = True)
            
        gdx_var = gdx_var.rename_axis(None, axis = 1)
        gdx_var = gdx_var.round(3)

        return(gdx_var)  

In [44]:
gdx_elec_cost = gdx_to_df(report_name = 'report_elec_cost',par_name = 'elec_cost')
display('gdx_elec_cost',gdx_elec_cost)
pv_pel_df = gdx_to_df(report_name = 'report_pv_pel',par_name = 'pv_pel')
display('pv_pel_df',pv_pel_df)
pv_share_df = gdx_to_df(report_name = 'report_pvshare',par_name = 'pv_share')
display('pv_share_df',pv_share_df)
bat_st_cap_df = gdx_to_df(report_name = 'report_bat_st_cap',par_name = 'bat_st_cap')
display('bat_st_cap_df',bat_st_cap_df)
gdx_stes_grid = gdx_to_df(report_name = 'report_grid_tot',par_name = 'val')
display('gdx_stes_grid',gdx_stes_grid)

full_scen_list = list(pd.unique(gdx_elec_cost['full_scen']))

'gdx_elec_cost'

,elec_cost,pv_capex,pv_opex,bat_capex,bat_opex,full_scen
0,1010.297,0.0,0.0,0.0,0.0,1030392_100056_PV_LiB_100000
1,1010.297,0.0,0.0,0.0,0.0,1030392_100032_PV_LiB_100000
2,2741.250,0.0,0.0,0.0,0.0,1030392_100128_PV_LiB_100000
3,2718.420,0.0,0.0,0.0,0.0,1030392_100147_PV_LiB_100000
4,627.609,0.0,0.0,0.0,0.0,1133034_100056_PV_LiB_100000
...,...,...,...,...,...,...
796,2832.365,0.0,0.0,0.0,0.0,1316897_100144_PV_LiB_807595
797,12989.274,0.0,0.0,0.0,0.0,1290392_100128_PV_LiB_819464
798,12920.524,0.0,0.0,0.0,0.0,1290392_100148_PV_LiB_819464
799,45994.549,0.0,0.0,0.0,0.0,1292871_100135_PV_LiB_863479


'pv_pel_df'

,full_scen,pv_pel
0,1030392_100056_PV_LiB_100000,0.0
1,1030392_100032_PV_LiB_100000,0.0
2,1030392_100128_PV_LiB_100000,0.0
3,1030392_100147_PV_LiB_100000,0.0
4,1133034_100056_PV_LiB_100000,0.0
...,...,...
796,1316897_100144_PV_LiB_807595,0.0
797,1290392_100128_PV_LiB_819464,0.0
798,1290392_100148_PV_LiB_819464,0.0
799,1292871_100135_PV_LiB_863479,0.0


'pv_share_df'

,full_scen,pv_share
0,1030392_100056_PV_LiB_100000,0.0
1,1030392_100032_PV_LiB_100000,0.0
2,1030392_100128_PV_LiB_100000,0.0
3,1030392_100147_PV_LiB_100000,0.0
4,1133034_100056_PV_LiB_100000,0.0
...,...,...
796,1316897_100144_PV_LiB_807595,0.0
797,1290392_100128_PV_LiB_819464,0.0
798,1290392_100148_PV_LiB_819464,0.0
799,1292871_100135_PV_LiB_863479,0.0


'bat_st_cap_df'

,full_scen,bat_st_cap
0,1030392_100056_PV_LiB_100000,0.0
1,1030392_100032_PV_LiB_100000,0.0
2,1030392_100128_PV_LiB_100000,0.0
3,1030392_100147_PV_LiB_100000,0.0
4,1133034_100056_PV_LiB_100000,0.0
...,...,...
796,1316897_100144_PV_LiB_807595,0.0
797,1290392_100128_PV_LiB_819464,0.0
798,1290392_100148_PV_LiB_819464,0.0
799,1292871_100135_PV_LiB_863479,0.0


'gdx_stes_grid'

,pv_prod_tot,sum_grid_dem_tot,full_scen
0,0.0,3282.317,1030392_100056_PV_LiB_100000
1,0.0,3282.317,1030392_100032_PV_LiB_100000
2,0.0,8905.946,1030392_100128_PV_LiB_100000
3,0.0,8831.773,1030392_100147_PV_LiB_100000
4,0.0,2039.015,1133034_100056_PV_LiB_100000
...,...,...,...
796,0.0,9201.965,1316897_100144_PV_LiB_807595
797,0.0,42200.371,1290392_100128_PV_LiB_819464
798,0.0,41977.010,1290392_100148_PV_LiB_819464
799,0.0,149429.985,1292871_100135_PV_LiB_863479


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Selecting policy scenario
</p><br>


In [45]:
print('rcp_scenario_year',rcp_scenario_year)
print('rcp_scen',rcp_scen)

rcp_scenario_year 2050
rcp_scen 8.5


In [46]:
fuel_scen_list = pd.unique(table_13['scenario']).tolist()
tax_list = pd.unique(table_14['tax_type']).tolist()
policy_scen_list = pd.unique(table_14['scenario']).tolist()
year_scen_list = rcp_scenario_year_list

In [47]:
print('fuel_scen_list',fuel_scen_list) 
print('tax_list',tax_list) 
print('policy_scen_list',policy_scen_list)
print('year_scen_list',year_scen_list) 

fuel_scen_list ['reference', 'high', 'low']
tax_list ['co2_tax', 'mos_tax', 'mwst_tax']
policy_scen_list ['bau', 'optimistic', 'ambitious']
year_scen_list [2020, 2025, 2030, 2035, 2040, 2045, 2050]


In [48]:
fuel_scen_widget = widgets.Dropdown(
    options = fuel_scen_list,
    description = 'Fuel scenario ',
    disabled = False)
display(fuel_scen_widget)

Dropdown(description='Fuel scenario ', options=('reference', 'high', 'low'), value='reference')

In [103]:
fuel_scen = fuel_scen_widget.value
fuel_scen

'high'

In [104]:
year_scen = rcp_scenario_year

In [78]:
policy_scen_widget = widgets.Dropdown(
    options = policy_scen_list,
    description = 'Policy scenario ',
    disabled = False)
display(policy_scen_widget)

Dropdown(description='Policy scenario ', options=('bau', 'optimistic', 'ambitious'), value='bau')

In [105]:
policy_scen = policy_scen_widget.value
policy_scen

'ambitious'

In [106]:
def syst_cost_func(fuel,fuel_demand_yr,price_band_arch,arch_entry_id):
        
    #print('fuel',fuel)                 

    #print('fuel_demand',fuel_demand)                 

    syst_cost_df = pd.DataFrame()

    price_df = table_5.copy()
    price_scen_df = table_13.copy().replace(np.nan, 0)
    tax_scen_df = table_14.copy()

    price_df = price_df[price_df.hvac_system == fuel]

    #print('price_df',price_df)                 

    price_band_list = list(pd.unique(price_df['price_band'].tolist()))

    #print('price_band_list',price_band_list)                 

    for price_band in price_band_list:

        lwr_fuel = float(price_df.loc[price_df['price_band'] == price_band, 'min_fuel'])

        hgr_fuel = float(price_df.loc[price_df['price_band'] == price_band, 'max_fuel'])

    #print('lwr_fuel',lwr_fuel,'hgr_fuel',hgr_fuel)

        if lwr_fuel <= fuel_demand_yr <= hgr_fuel:

            price_band_arch = price_band

    #print('price_band_arch',price_band_arch)                         

    base_price = price_df.loc[price_df['price_band'] == price_band_arch, 'base_price_chf_per_kWh'].item()
    base_mos = price_df.loc[price_df['price_band'] == price_band_arch, 'mos_chf_per_kWh'].item()
    base_co2 = price_df.loc[price_df['price_band'] == price_band_arch, 'co2_tax_chf_per_kWh'].item()
    base_mwst = price_df.loc[price_df['price_band'] == price_band_arch, 'mwst_%'].item()

    #print('base_price',base_price)                 
#     print('base_mos',base_mos,type(base_mos))                 
#     print('base_co2',base_co2,type(base_co2))                 
    #print('base_mwst',base_mwst)                 

    price_scen_df = price_scen_df[price_scen_df.hvac_system == fuel]
    price_scen_df = price_scen_df[price_scen_df.scenario == fuel_scen]

    price_scen_df = price_scen_df.drop(columns=['hvac_system','scenario'])

    base_price_df = (price_scen_df*base_price).astype(float)
    base_price_df = base_price_df.interpolate(method = 'linear',axis=1)\
                    .reset_index(drop=True).reset_index(drop = True)

    tax_scen_df = tax_scen_df[tax_scen_df.scenario == policy_scen]

    mos_tax_df = tax_scen_df[tax_scen_df.tax_type == 'mos_tax']
    mos_tax_df = mos_tax_df.drop(columns=['tax_type','scenario'])
    co2_tax_df = tax_scen_df[tax_scen_df.tax_type == 'co2_tax']
    co2_tax_df = co2_tax_df.drop(columns=['tax_type','scenario'])

    mos_tax_df = (mos_tax_df*base_mos).astype(float).reset_index(drop = True)
    co2_tax_df = (co2_tax_df*base_co2).astype(float).reset_index(drop = True)

    syst_cost_df = pd.DataFrame()

    #syst_cost_df = pd.concat([base_price_df, mos_tax_df,co2_tax_df]).reset_index()                 
    #syst_cost_df = syst_cost_df.sum(axis = 1)                 

    syst_cost_df = (base_price_df + mos_tax_df + co2_tax_df)*(1/(1-base_mwst)) 
    
#     if arch_entry_id == '1017477':
        
#         display(fuel,'base_price_df',base_price_df)                 
#         display(fuel,'mos_tax_df',mos_tax_df)                 
#         display(fuel,'co2_tax_df',co2_tax_df) 
#         display(fuel,'syst_cost_df',syst_cost_df)
#         display(fuel,'discount_df',discount_df)
    
#     display('syst_cost_df_final_price',syst_cost_df)
    
#     display('discount_df',discount_df)

    syst_cost_df = syst_cost_df*fuel_demand_yr
    
#     syst_cost_df_test = syst_cost_df.iloc[[0],[10,15]]

#     display('syst_cost_df_test',syst_cost_df_test) 

#     if arch_entry_id == '1017477':
#         display('final costs',syst_cost_df)

    return(syst_cost_df)
####END FUNCTION

In [107]:
def cost_elec_func(syst_elec_dict,table_2,table_4,wacc,start_yr,cost_type,tco_l_t):
    '''fuel_presel_dict is the result of the preselection process. The dictionary includes all archetypes 
    compatible with fuel based system solutions, namely oil, wood and gas. The preselection process filters 
    system solutions based on currently installed heating techbologies and its compatibility with possible 
    alternatives, considering power operation bands and geographic and technical limitations 
    (e.g. gas connection availability, borehole suitability, district heating in place, type of DHW system, etc )
    
    key: arch_entry_id
    value: list of all suitable technologies (syst_sol_id)'''
    
    cost_df = pd.DataFrame()  
    
    master_df = table_2.copy()
    syst_sol_df = table_4.copy()
    #need to add the STES data from gdx
    
    
    for arch_entry_id, syst_sol_list in syst_elec_dict.items():
        
        era_avg = master_df.loc[master_df['arch_entry_id'] == int(arch_entry_id), 'era_avg'].item()
        energy_specific = master_df.loc[master_df['arch_entry_id'] == int(arch_entry_id), 'energy_specific'].item()
        dhw_daily_energy = master_df.loc[master_df['arch_entry_id'] == int(arch_entry_id), 'dhw_daily_energy'].item()
        dhw_decentral = master_df.loc[master_df['arch_entry_id'] == int(arch_entry_id), 'dhw_decentral'].item()
#         inv_cost_m2 = master_df.loc[master_df['arch_entry_id'] == arch_entry_id, 'investment_cost__chf_per_m2'].item()
         
        
        for syst_sol in syst_sol_list:
            
            retro_id = arch_entry_id[-5:]
            
            if retro_id == 'retro':
                name_scenario = arch_entry_id[:-6] + '_' + str(syst_sol) + arch_entry_id[-6:]
            else:
                name_scenario = arch_entry_id + '_' + str(syst_sol)
            
            cost_df_loop = pd.DataFrame(np.array([name_scenario]),columns =['scenario'])            
            
            '''WACC dataframe calculation for the EAC calculation '''
            
            th_l_t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_l_t', syst_sol].item()           
                
            th_discount_df = discount_func(waac = wacc,start_yr = start_yr,lifetime = th_l_t)
                
            year_cols = list(discount_df.columns)
#             del year_cols[20]
            del year_cols[0]
#             print('year_cols',year_cols)
            
            '''CAPEX calculation'''
            th_capex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_capex_f', syst_sol].item()
#            print('th_capex_f',th_capex_f)     
            
            '''OPEX calculation'''
            
            th_opex_f = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_f', syst_sol].item()
#             print('th_opex_f',th_opex_f)
            
            th_opex_p = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_opex_p', syst_sol].item()
#             print('th_opex_p', th_opex_p)
            
            th_opex_cost_yr = th_opex_f  + (th_opex_p*th_capex_f)
        
            '''TCO OPEX calculation'''
            
            opex_df = th_discount_df * th_opex_cost_yr
#             print(name_scenario,th_opex_cost_yr,'opex_df', opex_df)
                  
            opex_cost_tco  = opex_df.iloc[0].sum()
#             print(scen,'opex_cost_tot',opex_cost_tot)
            
            
            '''Replacement cost calculation'''
            rep_cost_df = pd.DataFrame(np.array(['th_rc_1','th_rc_2','th_rc_3','th_rc_4','th_rc_5','total']),\
                                                 columns =['rep'])
            #print('rep_cost_df',rep_cost_df)
            
            
            th_rc_1 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1', syst_sol].item()
            th_rc_1t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_1t', syst_sol].item()
            
            th_rc_2 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2', syst_sol].item()
            th_rc_2t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_2t', syst_sol].item()
            
            th_rc_3 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3', syst_sol].item()
            th_rc_3t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_3t', syst_sol].item()
            
            th_rc_4 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4', syst_sol].item()
            th_rc_4t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_4t', syst_sol].item()
            
            th_rc_5 = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5', syst_sol].item()
            th_rc_5t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_rc_5t', syst_sol].item()
            
            th_rc_list = [th_rc_1,th_rc_2,th_rc_3,th_rc_4,th_rc_5]
            
            
            for year in year_cols:
                year_diff = year - year_cols[0] + 1
                rep_cost_list = []
                
                
                try:
                    rep_mod_1 = year_diff % th_rc_1t
                except ZeroDivisionError:
                    rep_mod_1 = 0.5
                try:
                    rep_mod_2 = year_diff % th_rc_2t
                except ZeroDivisionError:
                    rep_mod_2 = 0.5
                    
                try:    
                    rep_mod_3 = year_diff % th_rc_3t
                except ZeroDivisionError:
                    rep_mod_3 = 0.5
                    
                try:    
                    rep_mod_4 = year_diff % th_rc_4t
                except ZeroDivisionError:
                    rep_mod_4 = 0.5
                
                try:
                    rep_mod_5 = year_diff % th_rc_5t
                except ZeroDivisionError:
                    rep_mod_5 = 0.5
                
                rep_mod_list = [rep_mod_1,rep_mod_2,rep_mod_3,rep_mod_4,rep_mod_5]
                
                for rep_mod, th_rc in zip(rep_mod_list, th_rc_list):
                    
                    if rep_mod == 0:
                        
                        rep_cost = th_rc * th_discount_df.loc[0][year]
                        
                    else:
                        rep_cost = 0
                        
                    rep_cost_list.append(rep_cost)                  
                   
                rep_cost = sum(rep_cost_list)
                
                rep_cost_list.append(rep_cost)
                
                rep_cost_df[year] = rep_cost_list            
                        
#             print(rep_cost_df)
            
            rep_cost_tco = rep_cost_df.sum(axis = 1).get(key = 5)*0
#             print('rep_cost_total',rep_cost_total)
#             print(type(rep_cost_total)) 

            '''Calculating fuel & energy demand'''
    
            fuel_cost_eac = 0
            fuel_cost_tco = 0
    
            if syst_sol in th_dh_list or syst_sol in th_electro_list or syst_sol in th_wood_list\
            or syst_sol in th_gas_list or syst_sol in th_oil_list:
            
                th_systemtype_oil = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_oil', syst_sol].item()
                th_systemtype_gas = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_gas', syst_sol].item()
                th_systemtype_wood = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_systemtype_wood', syst_sol].item()

                th_dhw_decentral = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_dhw_decentral', syst_sol].item()
                th_efficiency = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_efficiency', syst_sol].item()
                th_l_t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_l_t', syst_sol].item()
                fuel_list = ['Oil','Gas','Wood']
                
                if dhw_decentral == 0: #energy_specific*era_avg

                    fuel_demand_yr = ((energy_specific * era_avg) + (dhw_daily_energy * 365)) / th_efficiency
                
                else:
                    fuel_demand_yr = (energy_specific * era_avg) / th_efficiency                
                
                if th_systemtype_oil == 1:
                
                    price_band_arch = 0

                    #Creating a one row dataframe, filtered by th_systemtype_oil
                    oil_cost_df = syst_cost_func(fuel = 'Oil',
                                                 fuel_demand_yr = fuel_demand_yr ,
                                                 price_band_arch = price_band_arch,
                                                 arch_entry_id = arch_entry_id)

                    #Calculating the EAC and TCO
                    fuel_cost_eac = oil_cost_df.loc[0,year_scen]
                    fuel_cost_tco = oil_cost_df.sum(axis = 1).get(key = 0)                
                
                elif th_systemtype_gas == 1:
                
                    price_band_arch = 0

                    #Creating a one row dataframe, filtered by th_systemtype_oil
                    gas_cost_df = syst_cost_func(fuel = 'Gas',
                                                 fuel_demand_yr = fuel_demand_yr ,
                                                 price_band_arch = price_band_arch,
                                                 arch_entry_id = arch_entry_id)

                    #Calculating the EAC and TCO
                    fuel_cost_eac = gas_cost_df.loc[0,year_scen]
                    fuel_cost_tco = gas_cost_df.sum(axis = 1).get(key = 0)

#                     display('arch_entry_id',arch_entry_id,gas_cost_df,'fuel_cost_eac',fuel_cost_eac)
            
                elif th_systemtype_wood == 1:

                    price_band_arch = 0
                    #Creating a one row dataframe, filtered by th_systemtype_oil
                    wood_cost_df = syst_cost_func(fuel = 'Wood',
                                                 fuel_demand_yr = fuel_demand_yr ,
                                                 price_band_arch = price_band_arch,
                                                 arch_entry_id = arch_entry_id)
                    
                    #Calculating the EAC and TCO
                    fuel_cost_eac = wood_cost_df.loc[0,year_scen]
                    fuel_cost_tco = wood_cost_df.sum(axis = 1).get(key = 0)

                else:
                    pass
                    
                
                        
#             '''Retrofit cost calculation'''
#             cost_retro = era_avg * inv_cost_m2                   
                    
            '''Adding the total costs to cost_df'''
            alpha_th = alpha_func(wacc = wacc,lifetime = th_l_t)     
            alpha_retro = alpha_func(wacc = wacc,lifetime = 30)
#             print('alpha_th',alpha_th)
            
            capex_eac = th_capex_f*alpha_th
                                
            opex_eac = th_opex_cost_yr
            
            repl_eac = 0
            
#             retro_eac = cost_retro*alpha_retro
            
            if cost_type == 'eac':
                
                cost_df_loop['ht_capex'] = capex_eac
                cost_df_loop['ht_opex'] = opex_eac
                cost_df_loop['replacement'] = repl_eac
                
#                 if retro_id == 'retro':
#                     cost_df_loop['retrofits'] = retro_eac
#                 else:
#                     cost_df_loop['retrofits'] = 0
                
                cost_df_loop['fuel'] = fuel_cost_eac
                    
                
            if cost_type == 'tco':
        
                cost_df_loop['ht_capex'] = th_capex_f
                cost_df_loop['ht_opex'] = opex_cost_tco
                cost_df_loop['replacement'] = rep_cost_tco
                
#                 if retro_id == 'retro':
#                     cost_df_loop['retrofits'] = cost_retro
#                 else:
#                     cost_df_loop['retrofits'] = 0
            
                cost_df_loop['fuel'] = fuel_cost_tco

            
            cost_df = cost_df.append(cost_df_loop,sort = False)   

    cost_df = cost_df.reset_index().drop(columns = 'index')
    
    '''Adding electricity and STES costs'''
    
    elec_cost_df = gdx_elec_cost.copy()
    
    scen_results_list = []
    stes_results_list = []
    stes_capex_list = []
    stes_vol_list = []
    elec_cost_list = []
    arch_list = []
    
    for full_scen in full_scen_list:
        arch = full_scen[:7]
        syst_sol = int(full_scen[8:14])
        scen_name = full_scen[:14]
        stes_id = int(full_scen[-6:])
#         stes_capex = table_4.loc[table_4['th_syssolution_id'] == 'th_capex_f', stes_id].item()
        stes_capex = gdx_stes_par.loc[gdx_stes_par['s'] == str(stes_id), 'stes_eac_capex'].item()
#         stes_l_t = table_4.loc[table_4['th_syssolution_id'] == 'th_l_t', stes_id].item()
        stes_l_t = 50
#         stes_vol = table_4.loc[table_4['th_syssolution_id'] == 'th_stes_volume', stes_id].item()
        stes_vol = gdx_stes_par.loc[gdx_stes_par['s'] == str(stes_id), 'th_stes_volume'].item()
        th_l_t = syst_sol_df.loc[syst_sol_df['th_syssolution_id'] == 'th_l_t', syst_sol].item()
        elec_cost = gdx_elec_cost.loc[elec_cost_df['full_scen'] == full_scen, 'elec_cost'].item()
        
        alpha_stes = alpha_func(wacc = wacc,lifetime = stes_l_t) 
        
        elec_cost_tot = (elec_cost*th_discount_df).iloc[0].sum()
        elec_cost_eac = elec_cost
        stes_cost_eac = stes_capex  
        stes_cost_tco = stes_capex/alpha_stes
        
        if cost_type == 'eac':
            stes_capex_list.append(stes_cost_eac)
            elec_cost_eac = elec_cost_eac
            elec_cost_list.append(elec_cost_eac)
             
        if cost_type == 'tco':
            stes_capex_list.append(stes_cost_tco)
            elec_cost_list.append(elec_cost_tot)
    
        arch_list.append(arch)
        scen_results_list.append(scen_name)
        stes_results_list.append(stes_id)
        stes_vol_list.append(stes_vol)
        
    
    elec_cost_df['arch_entry_id'] = arch_list
    elec_cost_df['scenario'] = scen_results_list
    elec_cost_df['stes'] = stes_results_list
    elec_cost_df['stes_vol'] = stes_vol_list
    elec_cost_df['stes_capex']= stes_capex_list
    elec_cost_df['elec_cost_tot']= elec_cost_list
    elec_cost_df = elec_cost_df.drop(columns = ['elec_cost'])
    
    elec_cost_df = pd.merge(elec_cost_df,gdx_stes_grid,on='full_scen')
    
    cost_df = pd.merge(elec_cost_df,cost_df,on='scenario')
    
    cost_df = pd.merge(cost_df,pv_pel_df,on='full_scen')
    cost_df = pd.merge(cost_df,bat_st_cap_df,on='full_scen')
    cost_df = pd.merge(cost_df,pv_share_df,on='full_scen')
    
    cost_df = cost_df.fillna(0).rename(columns={'pv_prod_tot':'pv_prod_yearly',\
                                          'sum_grid_dem_tot':'grid_dem_yearly'})
    
    cost_df = cost_df[['full_scen','arch_entry_id','scenario','stes','stes_vol','pv_pel','pv_share','bat_st_cap','pv_prod_yearly',\
                'grid_dem_yearly','elec_cost_tot','replacement','fuel','ht_capex','ht_opex','stes_capex',\
                'pv_capex','pv_opex','bat_capex','bat_opex']]
    
    sum_cols_list = ['elec_cost_tot','replacement','fuel','ht_capex','ht_opex','stes_capex',\
                     'pv_capex','pv_opex','bat_capex','bat_opex']

    cost_df['tot_cost'] = cost_df[sum_cols_list].sum(axis=1)
    
    cost_df.stes = cost_df.stes.astype(str)
    
#     cost_df = cost_df.drop(columns=['full_scen'])
                                          
    return(cost_df)

In [108]:
eac_elec_df = cost_elec_func(syst_elec_dict = syst_all_dict,\
                             table_2 = table_2,\
                             table_4 = table_4,
                             wacc = 0.04,\
                             start_yr = 2020,\
                             cost_type = 'eac',\
                             tco_l_t = 30)
eac_elec_df

,full_scen,arch_entry_id,scenario,stes,stes_vol,pv_pel,pv_share,bat_st_cap,pv_prod_yearly,grid_dem_yearly,elec_cost_tot,replacement,fuel,ht_capex,ht_opex,stes_capex,pv_capex,pv_opex,bat_capex,bat_opex,tot_cost
0,1030392_100056_PV_LiB_100000,1030392,1030392_100056,100000,0.000,0.0,0.0,0.0,0.0,3282.317,1010.297,0,1713.934175,7826.659478,2383.606353,0.000,0.0,0.0,0.0,0.0,12934.497006
1,1030392_100032_PV_LiB_100000,1030392,1030392_100032,100000,0.000,0.0,0.0,0.0,0.0,3282.317,1010.297,0,4423.419331,9303.137783,7728.841863,0.000,0.0,0.0,0.0,0.0,22465.695977
2,1030392_100128_PV_LiB_100000,1030392,1030392_100128,100000,0.000,0.0,0.0,0.0,0.0,8905.946,2741.250,0,0.000000,2034.461815,865.000000,0.000,0.0,0.0,0.0,0.0,5640.711815
3,1030392_100128_PV_LiB_203738,1030392,1030392_100128,203738,45.778,0.0,0.0,0.0,0.0,8737.916,2689.530,0,0.000000,2034.461815,865.000000,3155.541,0.0,0.0,0.0,0.0,8744.532815
4,1030392_100128_PV_LiB_814954,1030392,1030392_100128,814954,183.110,0.0,0.0,0.0,0.0,8644.112,2660.658,0,0.000000,2034.461815,865.000000,7671.387,0.0,0.0,0.0,0.0,13231.506815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,1292871_100135_PV_LiB_215869,1292871,1292871_100135,215869,194.326,0.0,0.0,0.0,0.0,150284.643,46257.613,0,0.000000,6086.633332,991.000000,7969.251,0.0,0.0,0.0,0.0,61304.497332
797,1292871_100135_PV_LiB_863479,1292871,1292871_100135,863479,777.302,0.0,0.0,0.0,0.0,149429.985,45994.549,0,0.000000,6086.633332,991.000000,19373.921,0.0,0.0,0.0,0.0,72446.103332
798,1292871_100156_PV_LiB_100000,1292871,1292871_100156,100000,0.000,0.0,0.0,0.0,0.0,151192.938,46537.186,0,0.000000,9395.355495,991.000000,0.000,0.0,0.0,0.0,0.0,56923.541495
799,1292871_100156_PV_LiB_215869,1292871,1292871_100156,215869,194.326,0.0,0.0,0.0,0.0,149925.564,46147.089,0,0.000000,9395.355495,991.000000,7969.251,0.0,0.0,0.0,0.0,64502.695495


In [109]:
# eac_elec_df = eac_elec_df.loc[eac_elec_df.arch_entry_id == '1206896']
# sum_cols_list = ['fuel','stes_capex','elec_cost_tot','ht_capex','ht_opex','replacement','pv_capex','pv_opex']

# eac_elec_df['tot_eac'] = eac_elec_df[sum_cols_list].sum(axis=1)
# eac_elec_df

In [110]:
# tco_elec_df = cost_elec_func(syst_elec_dict = syst_all_dict,\
#                              table_2 = table_2,\
#                              table_4 = table_4,\
#                              wacc = 0.04,\
#                              start_yr = 2020,\
#                              cost_type = 'tco',\
#                              tco_l_t = 20)
# tco_elec_df

In [111]:
# cost_dict = {}
# cost_dict['eac_elec_df'] = eac_elec_df
# cost_dict['tco_elec_df'] = tco_elec_df

In [112]:
def pd_to_csv(save_bool,save_dict,prefix,dir_csv):
    
    if save_bool == 'yes':
        for file_name, save_df in save_dict.items():

            out_csv =  prefix + file_name + '.csv' 

            if not os.path.exists(dir_csv):
                os.mkdir(dir_csv)

            full_dir_csv = os.path.join(dir_csv,out_csv)

            print(full_dir_csv)

            save_df.to_csv(full_dir_csv, sep = ';', index = False)

In [113]:
full_scen_name = data_results[:-18] + policy_scen + '_' + fuel_scen + '_' + pv_scen
full_scen_name

'2050_rcp85_Low_100_ambitious_high_pv_zro'

In [114]:
scen_eac_dict = {}
scen_eac_dict[full_scen_name] = eac_elec_df

In [115]:
pd_to_csv(save_bool = 'yes',
          save_dict = scen_eac_dict,
          prefix = 'eac_',
          dir_csv = './scen_results/eac/' + pv_scen + '/' )

./scen_results/eac/pv_zro/eac_2050_rcp85_Low_100_ambitious_high_pv_zro.csv


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Selecting the minimum cost system configuration per archetype
</p><br>

In [116]:
def min_arch(cost_df):
    tot_cost_df = cost_df.copy()
    
#     sum_cols_list = ['fuel','stes_capex','elec_cost_tot','ht_capex','ht_opex','replacement','pv_capex','pv_opex']
#     del_cols_list = ['fuel','stes_vol','stes_capex','pv_prod_yearly','grid_dem_yearly','elec_cost_tot','ht_capex',\
#                      'ht_opex','replacement','pv_capex','pv_opex']
                     
#     tot_cost_df['tot_cost'] = tot_cost_df[sum_cols_list].sum(axis=1)
#     tot_cost_df = tot_cost_df.drop(columns=del_cols_list)
    
    arch_type_list = []
    tot_building_list = []
    arch_list = tot_cost_df['arch_entry_id'].tolist()
    for arch_entry_id in arch_list:
        arch_type = table_2.loc[table_2['arch_entry_id'] == int(arch_entry_id), 'arch_typ'].item()
        tot_buildings = table_2.loc[table_2['arch_entry_id'] == int(arch_entry_id), 'tot_buildings'].item()
        arch_type_list.append(arch_type)
        tot_building_list.append(tot_buildings)
    
    tot_cost_df['arch_type'] = arch_type_list
    tot_cost_df['tot_buildings'] = tot_building_list
    tot_cost_df['pv_pel_tot'] = tot_cost_df['pv_pel']*tot_cost_df['tot_buildings']
    tot_cost_df['pv_pel_pot'] = (tot_cost_df['pv_pel'] / tot_cost_df['pv_share'])*tot_cost_df['tot_buildings']
    
    min_cost_df = pd.DataFrame()
    # tco_list=[min_tco_df,filt_tco_df]

    for arch in gdx_arch_list:
    
        filt_cost_df = tot_cost_df.loc[[tot_cost_df.loc[tot_cost_df.arch_entry_id == arch,'tot_cost'].idxmin()]]
        cost_list=[min_cost_df,filt_cost_df]
    
        min_cost_df = pd.concat(cost_list).reset_index(drop=True)
    
    min_scen_list = min_cost_df.scenario.tolist()
    min_hvac_syst_list = []
    
    for min_scen in min_scen_list:
        syst_sol = int(min_scen[-6:])
        hvac_system = table_4.loc[table_4['th_syssolution_id'] == 'th_heatingtechnology', syst_sol].item()
        min_hvac_syst_list.append(hvac_system)
        
    min_cost_df['hvac_system'] = min_hvac_syst_list
    
    min_cost_df  

    return(tot_cost_df,min_cost_df)

In [117]:
tot_eac_df,min_eac_df = min_arch(cost_df = eac_elec_df)
print('EAC')
display(min_eac_df)

# tot_tco_df, min_tco_df = min_arch(cost_df = tco_elec_df)
# print('TCO')
# display(min_tco_df)

EAC


,full_scen,arch_entry_id,scenario,stes,stes_vol,pv_pel,pv_share,bat_st_cap,pv_prod_yearly,grid_dem_yearly,elec_cost_tot,replacement,fuel,ht_capex,ht_opex,stes_capex,pv_capex,pv_opex,bat_capex,bat_opex,tot_cost,arch_type,tot_buildings,pv_pel_tot,pv_pel_pot,hvac_system
0,1030392_100128_PV_LiB_100000,1030392,1030392_100128,100000,0.0,0.0,0.0,0.0,0.0,8905.946,2741.250,0,0.0,2034.461815,865.0,0.0,0.0,0.0,0.0,0.0,5640.711815,SFH-Rural,1,0.0,NaN,hpaw
1,1133034_100125_PV_LiB_100000,1133034,1133034_100125,100000,0.0,0.0,0.0,0.0,0.0,5344.626,1645.076,0,0.0,1589.512971,865.0,0.0,0.0,0.0,0.0,0.0,4099.588971,SFH-Rural,3,0.0,NaN,hpaw
2,1107559_100130_PV_LiB_100000,1107559,1107559_100130,100000,0.0,0.0,0.0,0.0,0.0,16187.283,4982.446,0,0.0,2354.248102,865.0,0.0,0.0,0.0,0.0,0.0,8201.694102,SFH-Rural,4,0.0,NaN,hpaw
3,1132705_100127_PV_LiB_100000,1132705,1132705_100127,100000,0.0,0.0,0.0,0.0,0.0,8872.307,2730.896,0,0.0,1732.040821,865.0,0.0,0.0,0.0,0.0,0.0,5327.936821,SFH-Rural,1,0.0,NaN,hpaw
4,1069997_100126_PV_LiB_100000,1069997,1069997_100126,100000,0.0,0.0,0.0,0.0,0.0,8620.790,2653.479,0,0.0,1647.348226,865.0,0.0,0.0,0.0,0.0,0.0,5165.827226,SFH-Rural,1,0.0,NaN,hpaw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1300476_100125_PV_LiB_100000,1300476,1300476_100125,100000,0.0,0.0,0.0,0.0,0.0,10760.595,3312.111,0,0.0,1589.512971,865.0,0.0,0.0,0.0,0.0,0.0,5766.623971,SFH-Suburban,2,0.0,NaN,hpaw
93,1294820_100125_PV_LiB_100000,1294820,1294820_100125,100000,0.0,0.0,0.0,0.0,0.0,12178.156,3748.436,0,0.0,1589.512971,865.0,0.0,0.0,0.0,0.0,0.0,6202.948971,SFH-Rural,2,0.0,NaN,hpaw
94,1316897_100125_PV_LiB_100000,1316897,1316897_100125,100000,0.0,0.0,0.0,0.0,0.0,9327.264,2870.932,0,0.0,1589.512971,865.0,0.0,0.0,0.0,0.0,0.0,5325.444971,SFH-Suburban,10,0.0,NaN,hpaw
95,1290392_100128_PV_LiB_100000,1290392,1290392_100128,100000,0.0,0.0,0.0,0.0,0.0,42581.010,13106.435,0,0.0,2034.461815,865.0,0.0,0.0,0.0,0.0,0.0,16005.896815,MFH-Suburban,1,0.0,NaN,hpaw


In [118]:
min_cost_dict = {}
min_cost_dict[full_scen_name] = min_eac_df
# min_cost_dict['min_tco_df'] = min_tco_df

In [119]:
pd_to_csv(save_bool = 'yes',
          save_dict = min_cost_dict,
          prefix = 'min_' ,
          dir_csv = './scen_results/min_cost/' + pv_scen + '/' )

./scen_results/min_cost/pv_zro/min_2050_rcp85_Low_100_ambitious_high_pv_zro.csv


In [120]:
def tot_pv_and_bat(col_name):
    min_arch_type_list = list(pd.unique(min_eac_df['arch_type']))
    opt_df = pd.DataFrame()

    filt_df = min_eac_df.copy()

    filt_pv_df  = filt_df.drop(columns=['scenario','stes','arch_entry_id','tot_cost']) 
    sum_pv_list=[]

    for arch_type in min_arch_type_list:    
        arch_type_df = filt_df[filt_df.arch_type == arch_type]
        sum_pv = arch_type_df[col_name].sum()
        sum_pv_list.append(sum_pv)

    opt_df['arch_type'] = min_arch_type_list
    opt_df [col_name] = sum_pv_list

    return(opt_df)
    

In [121]:
opt_pv_elec_df = tot_pv_and_bat(col_name = 'pv_pel_tot')
pot_pv_elec_df = tot_pv_and_bat(col_name = 'pv_pel_pot')
opt_pv_elec_df = opt_pv_elec_df.merge(pot_pv_elec_df)
opt_pv_elec_df['pv_share'] = opt_pv_elec_df['pv_pel_tot'] / opt_pv_elec_df['pv_pel_pot']

display(opt_pv_elec_df)

,arch_type,pv_pel_tot,pv_pel_pot,pv_share
0,SFH-Rural,0.0,0.0,NaN
1,SFH-Suburban,0.0,0.0,NaN
2,MFH-Rural,0.0,0.0,NaN
3,MFH-Urban,0.0,0.0,NaN
4,SFH-Urban,0.0,0.0,NaN
5,MFH-Suburban,0.0,0.0,NaN


In [122]:
opt_bat_cap_df = tot_pv_and_bat(col_name = 'bat_st_cap')
display(opt_bat_cap_df)

,arch_type,bat_st_cap
0,SFH-Rural,0.0
1,SFH-Suburban,0.0
2,MFH-Rural,0.0
3,MFH-Urban,0.0
4,SFH-Urban,0.0
5,MFH-Suburban,0.0


In [123]:
tot_pv_pel_dict = {}
tot_pv_pel_dict[full_scen_name] = opt_pv_elec_df 

tot_bat_cap_dict = {}
tot_bat_cap_dict[full_scen_name] = opt_bat_cap_df

In [124]:
pd_to_csv(save_bool = 'yes',
          save_dict = tot_pv_pel_dict,
          prefix = 'pv_' ,
          dir_csv = './scen_results/tot_pv_pel/' + pv_scen + '/' )

pd_to_csv(save_bool = 'yes',
          save_dict = tot_bat_cap_dict,
          prefix = 'bat_' ,
          dir_csv = './scen_results/tot_bat_cap/' + pv_scen + '/')

./scen_results/tot_pv_pel/pv_zro/pv_2050_rcp85_Low_100_ambitious_high_pv_zro.csv
./scen_results/tot_bat_cap/pv_zro/bat_2050_rcp85_Low_100_ambitious_high_pv_zro.csv


In [125]:
hvac_type_list = ['hpaw','oil','gas','hpbw','wood']

def hvac_count():
#     hvac_unique_list = list(pd.unique(min_eac_df['hvac_system']))
    hvac_unique_list = ['hpaw','oil','gas','hpbw','wood']
    hvac_share_df = pd.DataFrame()

    filt_df = min_eac_df.copy()
    
    hvac_tot_count = len(filt_df.hvac_system.tolist())
    hvac_share_list = []
    
    for hvac in hvac_unique_list:
        hvac_df = filt_df[filt_df.hvac_system == hvac]
        hvac_share = len(hvac_df.hvac_system.tolist()) / hvac_tot_count
        hvac_share_list.append(hvac_share)
        
    hvac_share_df['hvac_system'] = hvac_unique_list
    hvac_share_df['hvac_share'] = hvac_share_list

    return(hvac_share_df)

In [126]:
hvac_share_df = hvac_count()
hvac_share_df

,hvac_system,hvac_share
0,hpaw,0.927835
1,oil,0.000000
2,gas,0.000000
3,hpbw,0.010309
4,wood,0.061856


In [127]:
hvac_share_dict = {}
hvac_share_dict[full_scen_name] = hvac_share_df

In [128]:
pd_to_csv(save_bool = 'yes',
          save_dict = hvac_share_dict,
          prefix = 'hvac_',
          dir_csv = './scen_results/hvac_share/' + pv_scen + '/')

./scen_results/hvac_share/pv_zro/hvac_2050_rcp85_Low_100_ambitious_high_pv_zro.csv


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
Aggregating electricity consumption profiles per type of archetype
</p><br>

In [ ]:
gdx_grid_dem_inst = gdxin.rgdx(name='report_grid_demand')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
gdx_grid_dem_inst

In [ ]:
# gdx_grid_dem_inst = gdxin.rgdx(name='report_grid_demand')
# gdx_stes_grid_spec = gdxin.rgdx(name='report_grid_demand')
# display('gdx_grid_dem_inst',gdx_grid_dem_inst)

gdx_grid_dem = pd.DataFrame(data=gdx_grid_dem_inst['values'].keys(), columns=gdx_grid_dem_inst['domain'])
gdx_grid_dem['val'] = gdx_grid_dem_inst['values'].values()
gdx_grid_dem = gdx_grid_dem.round(3) #rounding the values to 3 decimals
gdx_grid_dem.rename(columns = {'*':'cat'},inplace = True)
display('gdx_grid_dem',gdx_grid_dem)

In [ ]:
min_scen_list = min_eac_df['full_scen'].tolist()

In [ ]:
t_graph_list = list(pd.unique(gdx_grid_dem['t'].tolist()))
hy_list = []

for t in t_graph_list:
    hy = int(t[1:5])-1
    hy_list.append(hy)
    

In [ ]:
gdx_grid_dem
grid_cat_list = pd.unique(gdx_grid_dem['cat']).tolist()

In [ ]:
grid_cat_list

In [ ]:
gdx_min_dict = {}
grid_cat_list = pd.unique(gdx_grid_dem['cat']).tolist()
# display('grid_cat_list',grid_cat_list)

In [ ]:
for scen in min_scen_list:
    gdx_df = pd.DataFrame()

    gdx_scen = gdx_grid_dem.copy()
    gdx_scen = gdx_scen[gdx_scen.full_scen == scen]
    arch = int(scen[:7])
    tot_buildings = table_2.loc[table_2['arch_entry_id'] == arch, 'tot_buildings'].item()

    
    for cat in grid_cat_list:
        gdx_t = gdx_scen[gdx_scen.cat == cat].drop(columns = ['full_scen','cat','t'], axis=1).reset_index(drop = True)
        gdx_t.rename(columns={'val':cat}, inplace=True)
        gdx_df = pd.concat([gdx_df, gdx_t],axis = 1)
        
    gdx_df = gdx_df.astype(float)*tot_buildings
    gdx_df = gdx_df.drop(columns=['bat_disch','grid_dem_tot'])
                  
    gdx_min_dict[scen] = gdx_df
#     display('gdx_min_dict',gdx_min_dict)
 
gdx_min_dict       

In [ ]:
# def gdx_report(report_name,gdx_path,scen_graph_list):
    
#     gdx_rep_spec = gdxin.rgdx(name=report_name)
    
#     gdx_rep = pd.DataFrame(data=gdx_rep_spec['values'].keys(), columns=gdx_rep_spec['domain'])
#     gdx_rep['val'] = gdx_rep_spec['values'].values()
#     gdx_rep = gdx_rep.round(3) #rounding the values to 3 decimals
#     gdx_rep.rename(columns = {'*':'var'},inplace = True)   
    
#     gdx_dict = {}
    
#     for scen in scen_graph_list:
#         gdx_df = pd.DataFrame()

#         gdx_scen = gdx_rep.copy()
#         gdx_scen = gdx_scen[gdx_scen.full_scen == scen].drop(['full_scen'], axis=1)
        
    
#         for t in t_graph_list:
#             gdx_t = gdx_scen[gdx_scen.t == t].set_index('var').transpose().drop('t')
#             #gdx_t = gdx_t.rename_axis("t", axis="columns")

#             gdx_df = pd.concat([gdx_df, gdx_t], ignore_index = True)
        
#         if report_name == 'report_grid_demand':
#             gdx_df = gdx_df.drop(columns=['stes_ch_e3_el',\
#                                           'bat_ch','dhw_ch_el','bat_disch','grid_dem_tot'])
        
        
#         gdx_df = gdx_df.astype(float)
#         #gdx_df['t'] = t_graph_list
        
            
#         gdx_dict[scen] = gdx_df
        
#     return(gdx_dict)

In [ ]:
# gdx_stes_soc_dict = gdx_report(report_name ='report_stes_soc',\
#                                gdx_path = data_results_path,\
#                                scen_graph_list = full_scen_list)

In [ ]:
# gdx_stes_soc_dict

In [ ]:
# gdx_stes_grid_dict = gdx_report(report_name ='report_grid_demand',\
#                                gdx_path = data_results_path,\
#                                scen_graph_list = min_scen_list)

In [ ]:
# OJO
# gdx_stes_grid_dict = gdx_dict.copy()

In [ ]:
# gdx_stes_grid_dict

In [ ]:
min_tot_era_df = pd.DataFrame()

min_arch_type_list = []
min_roof_era_list = []
min_tot_era_list = []

for min_scen in min_scen_list:
    
    arch_entry_id = int(min_scen[:7])
    arch_type = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'arch_typ'].item()
    era_avg = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'era_avg'].item()
    roof_era = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'era_roof_new'].item()
    tot_buildings = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'tot_buildings'].item()
    tot_era = era_avg*tot_buildings
    roof_era_tot = roof_era*tot_buildings
    
    min_arch_type_list.append(arch_type)
    min_roof_era_list.append(roof_era_tot)
    min_tot_era_list.append(tot_era)
    
# min_tot_era_df['full_scen'] = min_scen_list
min_tot_era_df['arch_type'] = min_arch_type_list
min_tot_era_df['era_roof_new'] = min_roof_era_list
min_tot_era_df['tot_era'] = min_tot_era_list

min_tot_era_df = min_tot_era_df.groupby(by = 'arch_type',axis = 0,as_index = False).sum() 
   
display('min_tot_era_df',min_tot_era_df)


In [ ]:
sfh_urb_df = pd.DataFrame()
sfh_sub_df = pd.DataFrame()
sfh_rur_df = pd.DataFrame()
mfh_urb_df = pd.DataFrame()
mfh_sub_df = pd.DataFrame()
mfh_rur_df = pd.DataFrame()

for full_scen,grid_bal_df in gdx_min_dict.items():
    arch_entry_id = int(full_scen[:7])
    arch_type = table_2.loc[table_2['arch_entry_id'] == arch_entry_id, 'arch_typ'].item()
    
    if arch_type == 'SFH-Urban':
        sfh_urb_df = pd.concat([sfh_urb_df,grid_bal_df],axis = 1)
    elif arch_type == 'SFH-Suburban':
        sfh_sub_df = pd.concat([sfh_sub_df,grid_bal_df],axis = 1)
    elif arch_type == 'SFH-Rural':
        sfh_rur_df = pd.concat([sfh_rur_df,grid_bal_df],axis = 1)
    
    elif arch_type == 'MFH-Urban':
        mfh_urb_df = pd.concat([mfh_urb_df,grid_bal_df],axis = 1)
    elif arch_type == 'MFH-Suburban':
        mfh_sub_df = pd.concat([mfh_sub_df,grid_bal_df],axis = 1)
    elif arch_type == 'MFH-Rural':
        mfh_rur_df = pd.concat([mfh_rur_df,grid_bal_df],axis = 1)

# sfh_urb_df.to_csv('./scen_results/sfh_urb_df.csv', sep = ';', index = False)
sfh_urb_df = sfh_urb_df.groupby(level = 0, axis = 1).sum() 
sfh_sub_df = sfh_sub_df.groupby(level = 0, axis = 1).sum() 
sfh_rur_df = sfh_rur_df.groupby(level = 0, axis = 1).sum()

mfh_urb_df = mfh_urb_df.groupby(level = 0, axis = 1).sum() 
mfh_sub_df = mfh_sub_df.groupby(level = 0, axis = 1).sum() 
mfh_rur_df = mfh_rur_df.groupby(level = 0, axis = 1).sum() 
      
# display('sfh_urb_df',sfh_urb_df) 
# display('sfh_sub_df',sfh_sub_df)
# display('sfh_rur_df',sfh_rur_df) 
# display('mfh_urb_df',mfh_urb_df) 
# display('mfh_sub_df',mfh_sub_df) 
# display('mfh_rur_df',mfh_rur_df) 

arch_type_min_dict = {}
arch_type_min_dict['SFH-Urban'] = sfh_urb_df
arch_type_min_dict['SFH-Suburban'] = sfh_sub_df
arch_type_min_dict['SFH-Rural'] = sfh_rur_df
arch_type_min_dict['MFH-Urban'] = mfh_urb_df
arch_type_min_dict['MFH-Suburban'] = mfh_sub_df
arch_type_min_dict['MFH-Rural'] = mfh_rur_df

arch_type_min_dict





In [ ]:
full_scen_name

In [ ]:
arch_type_min_dict

In [ ]:
grid_dem_scen_df = pd.DataFrame()
pv_sup_scen_df = pd.DataFrame()
bat_ch_scen_df = pd.DataFrame()

# grid_dem_tot_df = pd.DataFrame()
# pv_sup_tot_df = pd.DataFrame()
# bat_ch_tot_df = pd.DataFrame()

for arch_type,bal_df in arch_type_min_dict.items():
    grid_dem_df = bal_df.drop(columns = ['bat_ch','dhw_cen_dem_el','dhw_dec_ch_el','pv_sup','heat_hp_el',\
                                   'stes_ch_e3_el','stes_ch_tot_hp_el'])
    pv_sup_df = bal_df.drop(columns = ['bat_ch','dhw_cen_dem_el','dhw_dec_ch_el','elec_dem','heat_hp_el',\
                                   'stes_ch_e3_el','stes_ch_tot_hp_el'])
    bat_ch_df = bal_df.drop(columns = ['pv_sup','dhw_cen_dem_el','dhw_dec_ch_el','elec_dem','heat_hp_el',\
                                   'stes_ch_e3_el','stes_ch_tot_hp_el'])
    
    grid_dem_scen_df = pd.concat([grid_dem_scen_df,grid_dem_df],axis = 1)
    pv_sup_scen_df = pd.concat([pv_sup_scen_df,pv_sup_df],axis = 1)
    bat_ch_scen_df = pd.concat([bat_ch_scen_df,bat_ch_df],axis = 1)
    
grid_dem_scen_df = grid_dem_scen_df.groupby(level = 0, axis = 1).sum()
pv_sup_scen_df = pv_sup_scen_df.groupby(level = 0, axis = 1).sum()
bat_ch_scen_df = bat_ch_scen_df.groupby(level = 0, axis = 1).sum()

grid_dem_scen_df.rename(columns={'elec_dem':full_scen_name}, inplace=True)
pv_sup_scen_df.rename(columns={'pv_sup':full_scen_name}, inplace=True)
bat_ch_scen_df.rename(columns={'bat_ch':full_scen_name}, inplace=True)

grid_dem_tot_df = pd.concat([grid_dem_tot_df,grid_dem_scen_df],axis = 1)
pv_sup_scen_df = pd.concat([pv_sup_scen_df,pv_sup_scen_df],axis = 1)
bat_ch_tot_df = pd.concat([bat_ch_tot_df,bat_ch_scen_df],axis = 1)

    
print(bat_ch_tot_df)

In [ ]:
'''GRIMSEL'''

In [ ]:
arch_type_tot_dict = {}
tot_pv_elec_df = pd.DataFrame()
tot_pv_elec_list = []
arch_type_pv_list = []

for arch_type,arch_df in arch_type_min_dict.items():
    
    arch_df_pv = pd.DataFrame()
    
    min_tot_era = min_tot_era_df.loc[min_tot_era_df['arch_type'] == arch_type, 'tot_era'].item()
    tot_era_avg = table_12.loc[table_12['arch_typ'] == arch_type, 'tot_era_avg'].item()
    min_roof_era = min_tot_era_df.loc[min_tot_era_df['arch_type'] == arch_type, 'era_roof_new'].item()
    tot_roof_era = table_12.loc[table_12['arch_typ'] == arch_type, 'era_roof_new'].item()
    
    arch_df_pv = arch_df['pv_sup']
    arch_df_pv = (arch_df_pv/min_roof_era)*(tot_roof_era)
    
    arch_df = arch_df.drop(columns = ['pv_sup'])
    
    arch_df = arch_df.div(min_tot_era).mul(tot_era_avg)
    
    arch_df = pd.concat([arch_df,arch_df_pv],axis = 1)
#     era_col_list = ['bat_ch', 'dhw_cen_dem_el','dhw_ch_el','elec_dem','heat_hp_el','stes_ch_e3_el','stes_ch_tot_hp_el']
   
    arch_df['hy'] = hy_list
    
    arch_type_tot_dict[arch_type] = arch_df
    
    opt_pv_elec = opt_pv_elec_df.loc[opt_pv_elec_df['arch_type'] == arch_type, 'pv_pel_tot'].item()
    
    arch_type_pv_list.append(arch_type) 
    
    tot_pv_elec_list.append((opt_pv_elec/min_roof_era)*(tot_roof_era))
    
#     print('tot_roof_era',tot_roof_era)
    
tot_pv_elec_df['arch_type'] = arch_type_pv_list  
tot_pv_elec_df['tot_pv_elec'] = tot_pv_elec_list

arch_type_tot_dict

In [ ]:
tot_pv_elec_df

In [ ]:
tot_pv = tot_pv_elec_df['tot_pv_elec'].sum()
tot_pv

In [ ]:
for arch_type, grid_bal_df in arch_type_tot_dict.items():
    

    out_csv = arch_type + '.csv' 
    
    #dir_csv = dir_parsed + '/' + arch 
    
    dir_csv = './scen_results' + 'GRIMSEL'
    
    #dir_cv = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/plots/10MFH/PVmax'
    print(dir_csv)
    
    if not os.path.exists(dir_csv):
        os.mkdir(dir_csv)
        
    full_dir_csv = os.path.join(dir_csv,out_csv)
    
    print(full_dir_csv)
    
    arch_type_tot_dict[arch_type].to_csv(full_dir_csv, sep = ';', index = False)
 

In [ ]:
out_csv = 'tot_pv_pel.csv' 
   
#dir_csv = dir_parsed + '/' + arch 
    
dir_csv = './10MFH/' + 'GRIMSEL'
    
#dir_cv = 'C:/Users/xigeby65/switchdrive/SwissStore/Model/Results/plots/10MFH/PVmax'
print(dir_csv)
    
if not os.path.exists(dir_csv):
    os.mkdir(dir_csv)
        
full_dir_csv = os.path.join(dir_csv,out_csv)
    
print(full_dir_csv)
    
tot_pv_elec_df.to_csv(full_dir_csv, sep = ';', index = False)